<a href="https://colab.research.google.com/github/lunaczp/stanford_alpaca/blob/main/train_in_colab_opt-125m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train alpaca in colab

In [34]:
! nvidia-smi

Sun May 26 09:29:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              66W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [9]:
! git clone https://github.com/lunaczp/stanford_alpaca.git

Cloning into 'stanford_alpaca'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 145 (delta 6), reused 12 (delta 4), pack-reused 129
Receiving objects: 100% (145/145), 9.15 MiB | 21.49 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [15]:
%cd /content/stanford_alpaca/
! which python pip
! which python3.10 pip3.10

/content/stanford_alpaca
/usr/local/bin/python
/usr/local/bin/pip
/usr/bin/python3.10
/usr/local/bin/pip3.10


In [30]:
! pip3.10 install -r requirements.txt
! pip3.10 install accelerate -U

Name: wandb
Version: 0.17.0
Summary: A CLI and library for interacting with the Weights & Biases API.
Home-page: 
Author: 
Author-email: Weights & Biases <support@wandb.com>
License: MIT License
        
        Copyright (c) 2021 Weights and Biases, Inc.
        
        Permission is hereby granted, free of charge, to any person obtaining a copy
        of this software and associated documentation files (the "Software"), to deal
        in the Software without restriction, including without limitation the rights
        to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
        copies of the Software, and to permit persons to whom the Software is
        furnished to do so, subject to the following conditions:
        
        The above copyright notice and this permission notice shall be included in all
        copies or substantial portions of the Software.
        
        THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
        IM

In [32]:
# fix wandb.errors not found, (the wandb in dist-packages is not complete for unknown reason)
! pip3.10 uninstall wandb
! pip3.10 install wandb
! pip3.10 show wandb
import wandb
print(wandb.__version__)

Found existing installation: wandb 0.17.0
Uninstalling wandb-0.17.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/package_readme.md
    /usr/local/lib/python3.10/dist-packages/wandb-0.17.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/wandb/*
Proceed (Y/n)? 
  Successfully uninstalled wandb-0.17.0
  Using cached wandb-0.17.0-py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.7 MB)
Name: wandb
Version: 0.17.0
Summary: A CLI and library for interacting with the Weights & Biases API.
Home-page: 
Author: 
Author-email: Weights & Biases <support@wandb.com>
License: MIT License
        
        Copyright (c) 2021 Weights and Biases, Inc.
        
        Permission is hereby granted, free of charge, to any person obtaining a copy
        of this software and associated documentation files (the "Software"), to deal
        in the Software without restriction, including without limitation the rights
        to use, copy, m

In [21]:
import torch
c =  torch.cuda.is_available();
d = torch.device("cuda")
print(d)

# Check the number of available GPUs
num_gpus = torch.cuda.device_count()
print(f"Number of GPUs available: {num_gpus}")

cuda
Number of GPUs available: 1


In [37]:
#! export CUDA_VISIBLE_DEVICES=0
# nproc_per_node 有几个GPU，就设置几
# model_name_or_path
#   facebook/opt-6.7b
#   facebook/opt-125m
! torchrun --nproc_per_node=1 --master_port=10011 train.py \
    --model_name_or_path "facebook/opt-125m" \
    --data_path ./alpaca_data.json \
    --bf16 True \
    --output_dir output \
    --num_train_epochs 3 \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 8 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 2000 \
    --save_total_limit 1 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True
    #--fsdp "full_shard auto_wrap" \
    #--fsdp_transformer_layer_cls_to_wrap 'OPTDecoderLayer' \


2024-05-26 09:38:28.053547: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-26 09:38:28.103588: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 09:38:28.103634: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 09:38:28.104904: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-26 09:38:28.111947: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
! mkdir model
# mv model file to model
! mv output/* model/
! mv model/runs output/
! mv model/checkpoint-* output/

In [43]:
# test
from transformers import AutoTokenizer, OPTForCausalLM

model = OPTForCausalLM.from_pretrained("./model/", local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained("./model/", local_files_only=True)

prompt = "Write a poem:"

inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'Write a poem:\n\nThe sun rises,\nA peaceful night sky,\nA peaceful place to rest.\n\nThe stars twinkle'

In [44]:
! pip3.10 install -U "huggingface_hub[cli]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 889.7 kB/s eta 0:00:00


In [45]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

In [46]:
%cd model
! huggingface-cli upload LightXXXXX/opt-125m-alpaca-light . .

/content/stanford_alpaca/model
Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
model.safetensors:   0% 0.00/501M [00:00<?, ?B/s]
Upload 2 LFS files:   0% 0/2 [00:00<?, ?it/s]

training_args.bin:   0% 0.00/5.11k [00:00<?, ?B/s]

training_args.bin: 100% 5.11k/5.11k [00:00<00:00, 19.7kB/s]
model.safetensors: 100% 501M/501M [00:09<00:00, 50.6MB/s]

Upload 2 LFS files: 100% 2/2 [00:10<00:00,  5.08s/it]
https://huggingface.co/LightXXXXX/opt-125m-alpaca-light/tree/main/.
